In [1]:
import sys
import stablemotifs
import biolqm
import ginsim
import pystablemotifs as sm
import pyboolnet
import boolsim
import timeit
import timeout_decorator
from timeout_decorator import TimeoutError
import pandas as pd
import glob
from colomoto_jupyter import tabulate
import numpy as np


WARNING In /home/david/anaconda2/envs/python_3_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
WARNING In /home/david/anaconda2/envs/python_3_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
WARNING In /home/david/anaconda2/envs/python_3_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
WARNING In /home/david/anaconda2/envs/python_3_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and wi

## Parameters of the bechmark

In [2]:

#the folder in which all models we whish to run are saved in the booleanned format with .booleannet extention. 
models_path='models_for_benchmark/'

#run_ens_size is the number of times each method is repeatedly run IN THE SAME ITERATION. 
#this means that the timeout has to be increased proporionally to account for all runs
#In the final output the minimum (best) runtime is saved out of all in the ensemble 
run_ens_size=1 

#timeout limit in seconds for each method (all runs within the run_ens_size range). 
#If the timeout limit is reached the output will be -1
timeout_limit=43200


#name of methods to be exluded from the run. This can be set manually if we know 
#that some methods are not relevant or take too much time
exclude_from_run=set([])
#uncomment the line abovr if you want only methods that look for complex attractors

#the excel file in which the dataframe summarizing the results will be saved.
#if it already exists new results will be added to it
output_file_name='timing_results.xlsx'


In [3]:
@timeout_decorator.timeout(timeout_limit)
def run_timed_test(method_runstring, setup, repeat):

    return timeit.repeat(method_runstring,setup=setup, repeat=repeat, number=1)

try: #if the file already exists we dont overwrite it
    result_df=pd.read_excel(output_file_name)
    result_df=result_df.reset_index(drop=True)
except FileNotFoundError:
    result_df=pd.DataFrame()
    
#grabbing all the .booleannet models from the folder
model_set=glob.glob(models_path+'*.booleannet')
#model_set=['PSO.txt', 'phase_switch.txt','myeloid.txt'] 

for model_file_name in model_set:
    print('\nRunning model %s'%model_file_name)
    
    base_setup_string='''import stablemotifs
import biolqm
import ginsim
import pystablemotifs as sm
import pyboolnet
import boolsim
import cabean
import pypint
import json
from colomoto_jupyter import tabulate


model_file_name='%s'
with open(model_file_name) as f:
    rules=f.read().replace('#BOOLEAN RULES','').strip()
biolqm_model = biolqm.load(model_file_name, "booleannet")
pypint_model=biolqm.to_pint(biolqm_model)'''%model_file_name

    method_runstrings={'stablemotifs2013': "a=stablemotifs.load(biolqm_model, quiet=True).attractors\ntabulate(a).to_excel('attr_output/attr_output_stablemotifs2013_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'pystablemotifs': "ar = sm.AttractorRepertoire.from_primes(primes, max_simulate_size=max_simulate_size)\nex.attractor_dataframe(ar).to_excel('attr_output/attr_output_pystablemotifs_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'pyboolnet': 'd=compute_attractors(primes, "asynchronous")\nwith open(\'attr_output/attr_output_PyBoolNet_\'+model_file_name.split(\'.\')[0].split(\'/\')[-1]+\'.json\',\'w\') as fp:\n\tjson.dump(d,fp)',
                   
                   
 'boolsim': "a=boolsim.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_boolsim_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'bioLQM': "a=biolqm.fixpoints(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_bioLQM_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                                      
                   
 'CABEAN': "a=cabean.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_CABEAN_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'Pint': "a=pypint.fixpoints(pypint_model)\ntabulate(a).to_excel('attr_output/attr_output_Pint_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')"}

    method_setups={'stablemotifs2013':base_setup_string,

    'pystablemotifs':base_setup_string+'''
max_simulate_size=20
rules_pbn = sm.format.booleannet2bnet(rules)
primes = pyboolnet.prime_implicants.bnet_text2primes(rules_pbn)
import pystablemotifs.export as ex
''',

    'pyboolnet':base_setup_string+'''
rules_pbn = sm.format.booleannet2bnet(rules)
primes = pyboolnet.prime_implicants.bnet_text2primes(rules_pbn)
from pyboolnet.attractors import compute_attractors
''',

    'boolsim':base_setup_string,

    'bioLQM':base_setup_string,

    'CABEAN':base_setup_string,

    'Pint':base_setup_string}

    method_runtimes={i:[] for i in method_runstrings}
    
    
    for method_name in set(method_runstrings)-exclude_from_run:
        print('method:',method_name)
        try:
            method_runtimes[method_name]=run_timed_test(method_runstrings[method_name], setup=method_setups[method_name], repeat=run_ens_size)
        except Exception as e:
            print(str(e))
            method_runtimes[method_name].append(-1)
    runtime_minimums={k:min(method_runtimes[k]) for k in method_runtimes.keys()-exclude_from_run} 
    runtime_minimums['model']=model_file_name.split('.')[0].split('/')[-1]
    result_df=result_df.append([runtime_minimums],ignore_index=True)
    result_df=result_df.reset_index(drop=True)

    result_df.to_excel(output_file_name)


Running model models_for_benchmark/phase_switch.booleannet
method: pyboolnet


INFO attractors.completeness(..)
INFO created /tmp/pyboolnet_aux275pd
INFO yes
INFO  working on minimal trapspace 1/3: 01100110110
INFO attractors.univocality(..)
INFO  yes
INFO attractors.faithfulness(..)
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=11, update=asynchronous, length=110, attempts=10
INFO trial 0
INFO start: 01100110110
INFO end: 01100110110
INFO created /tmp/pyboolnet_e51kow7v
INFO is attractor state
INFO  working on minimal trapspace 2/3: 00010000010
INFO attractors.univocality(..)
INFO  yes
INFO attractors.faithfulness(..)
INFO yes
INFO attractors.find_attractor_state_by_randomwalk_and_ctl(..)
INFO find_attractor_state_by_randomwalk_and_ctl(..)
INFO len(primes)=11, update=asynchronous, length=110, attempts=10
INFO trial 0
INFO start: 00010000010
INFO end: 00010000010
INFO created /tmp/pyboolnet_bt1ychpu
INFO is attractor state
INFO  working on minimal trapspace 3/3: 0010101

In [4]:
result_df

,pyboolnet,Pint,boolsim,stablemotifs2013,bioLQM,CABEAN,pystablemotifs,model
0,0.57275,0.354752,0.257168,59.296916,0.188039,0.178952,0.93527,phase_switch
